In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
import expectexception

# Iterators, Generators


Python's composite data structures can be used in for loops and comprehensions.  Other objects can be used in these too, as long as they support the iterable and iterator interfaces.

One convenient way to do this is by creating a generator.  A generator acts like a function that pauses every time it reaches the `yield` keyword.  Coroutines are related construction that allow the function to pause for input.  While a little abstract, these tools allow for more readable code overall by separating concerns.

## Iterables and Iterators


What's happening inside a for loop?

In [3]:
for i,item in enumerate([1, 2, 3]):
    print(i,item)

0 1
1 2
2 3


Python is not actually figuring out the length of the list and then indexing through it.  Instead, it is doing something akin to the following:

In [4]:
it = iter([1, 2, 3])
while True:
    try:
        i = next(it)
    except StopIteration:
        break
    print(i)

1
2
3


The list is an **iterable**.  That means we can call `iter()` on it and get an **iterator**.

In [5]:
type(it)

list_iterator

This iterator will go through the list *once* and then raise `StopIteration`.  After that, it cannot be reused.

In [6]:
try:
    next(it)
except StopIteration:
    print("Iterator exhausted")

Iterator exhausted


The `iter()` function just calls the `__iter__()` method of its argument.  Therefore, implementing an iterable is as simple as giving a class an `__iter__()` method.

This is a good example of Python's "duck typing".  There's no iterable class or interface to inherit from.  It just checks if there is a `__iter__()` method.  If so, that's good enough to be an iterable.

In [8]:
class Iterable(object):
    
    def __init__(self, n):
        self.n = n
    
    def __iter__(self):
        return iter([self.n] * self.n)

In [9]:
my_iter = Iterable(5)
for i in my_iter:
    print(i)

5
5
5
5
5


Similarly, all it takes to be an iterator is a `__next__()` method.

In [10]:
class Iterator(object):
    
    def __init__(self, n):
        self.curr = n + 1
    
    def __next__(self):
        self.curr -= 1
        if self.curr >= 0:
            return self.curr
        else:
            raise StopIteration

This iterator isn't an iterable, so we can't use it directly in a for loop.  We can demo it in the same while loop we used before.

In [11]:
iterator = Iterator(5)
while True:
    try:
        i = next(iterator)
    except StopIteration:
        break
    print(i)

5
4
3
2
1
0


To actually use it in a for loop, we'd need an iterable that returns it.  We could make a separate class, but it's easier to just make the iterator be iterable as well.

In [12]:
class IterableIterator(object):
    
    def __init__(self, n):
        self.curr = n + 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.curr -= 1
        if self.curr >= 0:
            return self.curr
        else:
            raise StopIteration

In [13]:
for i in IterableIterator(5):
    print(i)

5
4
3
2
1
0


## Generators

Generators are a type of iterator.  Benefits:
1. They are more powerful than just using `map` and `filter` because they allow you to hold state in between processing entries.  They are like `reduce` but much easier to use, which makes them powerful.
1. They allow you to hold data in an "inner" context without needing to resort to creating a `class`.  This can be faster since `self.foo` is actually pretty slow in python.
1. **Gotcha**: the generator is not run until you first call `.__next__`, which can be a bit counter-intuitive ...

Here's a generator that does the same countdown as the `IterableIterator` we defined above.  Notice that, even with some print statements, it uses fewer lines of code.

In [14]:
def Countdown(n):
    print("Counting down ...")
    while n > 0:
        yield n
        n -= 1

c = Countdown(5)
print("Set up Countdown")
for i in c:
    print(i)

Set up Countdown
Counting down ...
5
4
3
2
1


`Countdown` is a generator function.  When it is called, it immediately returns a generator object, but no code is executed.

In [15]:
c = Countdown(3)
c

<generator object Countdown at 0x00000225647500B0>

Generators are iterators, so we can use the `next` method to allow the execution to proceed to the next `yield` statement.

In [16]:
my_list=[1,2,3,4,6,10,100]
#[i for i,item in enumerate(my_list) if item>5]

In [17]:
print(next(c))
print(next(c))

Counting down ...
3
2


When the generator returns during a `.__next__()` call, `StopIteration` is raised.  This is used by for loops and list comprehensions to signal the end of the iterator.

## Generator "pipelines"


In particular, we're going to create this generator

```
source_gen -> and_plus_one_gen -> sum_gen
```

and chain them together.  Note that for each generator input, we can yield none, one, or multiple outputs.

1. **Source:** pushes values using `yield`.
2. **Intermediate Step:** both requests previous values (`next`) and pushes them using `yield`
3. **Sink:** iterates through previous values using `next`.

**Question:** why is this better than dealing with a list?

In [20]:
def source_gen(n):
    for i in range(n):
        yield i

def and_plus_one_gen(gen):
    for i in gen:
        yield i
        yield i + 1

def sum_gen(gen):
    return sum(i for i in gen)

gen1 = source_gen(3)
gen2 = and_plus_one_gen(gen1)
result = sum_gen(gen2)
result

9

## Generator comprehensions


Python supports generator comprehensions in addition to list comprehensions.  They use parentheses `()` instead of brackets `[]`.  While concise, they can only do `map` and `filter`-like things.

In [21]:
(j for j in range(10))

<generator object <genexpr> at 0x00000225647505F0>

Actually, the parentheses are needed only to group the expression together.  If the generator comprehension appears in another set of parentheses, a second set is not needed.

In [24]:
sum(j for j in range(4))

6

Comprehensions can be nested.  This produces a flattened list or generator.  Note that the syntax reads outwards from the middle: The outermost loop appears in the middle, the inner loop appears at the end, and the quantity calculated in that loop appears at the beginning.

In [25]:
[j for i in range(10) for j in (i, i+1)]

[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10]

Using this, we can reproduce the previous generator pipeline.

In [26]:
sum(j for i in range(10) for j in (i, i+1))

100

### Not all generators can be written as generator comprehensions


It might seem from the above example that all generators can be written as generator expressions.  This is not true.  Generator expressions cannot keep track of state in between processing elements, generators can.  In the following example, the `total` variable holds state between generator iterations.

In [27]:
def and_total_gen(gen):
    total = 0
    for i in gen:
        yield i
        total += i
        yield total

In [28]:
list(and_total_gen(range(10)))

[0, 0, 1, 1, 2, 3, 3, 6, 4, 10, 5, 15, 6, 21, 7, 28, 8, 36, 9, 45]

## Time complexity


Because they don't have to construct an entire list, iterators are much faster. Generator comprehensions will be faster than list comprehensions. They are also much more memory efficient (typically `O(1)` rather than `O(n)`).

In [29]:
%%timeit -n1
my_list=[1,2,3,4,5,6,7,8,9]
def fi_con(list):
    it = iter(list)
    count = 0
    while count < len(list):
        item = next(it)
        if item > 9:
            return count
        else:
            count += 1

The slowest run took 6.00 times longer than the fastest. This could mean that an intermediate result is being cached.
414 ns ± 323 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit -n1
my_list=[1,2,3,4,5,6,7,8,9]
for idx,item in zip(range(len(my_list)),my_list):
    if item > 5:
        print(idx)
        break

5
5
5
5
5
5
5
The slowest run took 26.85 times longer than the fastest. This could mean that an intermediate result is being cached.
84.1 µs ± 152 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit -n1
my_list=[1,2,3,4,5,6,7,8,9]
print(next(i for i,item in enumerate(my_list) if item>5))

5
5
5
5
5
5
5
The slowest run took 4.54 times longer than the fastest. This could mean that an intermediate result is being cached.
19.9 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Itertools in Python


Manipulating iterators requires a little more care than before.  The itertools package provides some nice tools for dealing with iterators.  Remember that many common operations like `range` and `dict.items` are returning iterators and not lists.

In [32]:
from itertools import count, islice, chain, tee, takewhile, dropwhile, combinations

Count is an iterator that is never exhausted.  We can slice a portion of it.

In [33]:
list(islice(count(), 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [34]:
list(chain(range(10), range(10)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [38]:
it = range(5)
it1, it2 = tee(it, 2)
list(it1)

[0, 1, 2, 3, 4]

In [39]:
list(it2)

[0, 1, 2, 3, 4]

In [40]:
list(it1)

[]

In [41]:
list(takewhile(lambda x: x < 'C', 'ABCDABCD'))

['A', 'B']

In [42]:
list(dropwhile(lambda x: x < 'C', 'ABCDABCD'))

['C', 'D', 'A', 'B', 'C', 'D']

In [43]:
list(combinations(range(4), 2))

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]